In [1]:
import xarray as xr 

In [2]:
vil_ds = xr.open_zarr('../hrrr17x.zarr/')
vil_ds

<xarray.Dataset>
Dimensions:  (id: 5625, lat: 25, lon: 25)
Dimensions without coordinates: id, lat, lon
Data variables:
    VIL      (id, lat, lon) float32 dask.array<chunksize=(1407, 7, 13), meta=np.ndarray>

In [3]:
vil_dr = vil_ds.VIL

# Traffic Setup

In [4]:
# Begin downloading traffic data corresponding to the thunderstorm dates indicated by vil.attrs['date']
from traffic.data import opensky
opensky.username='thinhhoang'
opensky.password='iQ6^yrwe7o3m'
from datetime import datetime, timedelta 
import pytz 

dallas_tz = pytz.timezone('US/Central')
paris_tz = pytz.timezone('Europe/Paris')
utc_tz = pytz.timezone('UTC')

Some Helper functions to handle datetime

In [5]:
def str_to_datetime_utc(s):
    return utc_tz.localize(datetime.strptime(s, '%Y-%m-%d %H:%M:%S'))

def to_dallas_tz(dt):
    return dt.astimezone(dallas_tz)

def to_paris_tz(dt):
    return dt.astimezone(paris_tz)

def to_utc_tz(dt):
    return dt.astimezone(utc_tz)

def to_string_from_datetime(dt):
    return dt.strftime('%Y-%m-%d %H:%M:%S')

Merge multiple time entries together

In [6]:
def merge_contiguous_storms(datetime_list):
    # Convert to datetime
    datetime_list = [str_to_datetime_utc(dt) for dt in datetime_list]
    
    # Sort the list
    datetime_list.sort()
    
    # Merge contiguous storms
    merged_datetime_list = [] # end of the storm
    merged_datetime_list_begin = [] # beginning of the storm
    
    for dt in datetime_list:
        if len(merged_datetime_list) == 0:
            merged_datetime_list.append(dt)
            merged_datetime_list_begin.append(dt)
        else:
            if dt - merged_datetime_list[-1] > timedelta(hours=1):
                merged_datetime_list.append(dt)
                merged_datetime_list_begin.append(dt)
            else:
                merged_datetime_list[-1] = dt
                
    # Convert back to string
    merged_datetime_list = [to_string_from_datetime(dt) for dt in merged_datetime_list]
    merged_datetime_list_begin = [to_string_from_datetime(dt) for dt in merged_datetime_list_begin]
    
    return merged_datetime_list_begin, merged_datetime_list

In [7]:
storm_catalogue = merge_contiguous_storms(vil_dr.attrs['date']) # merged contiguous storms
storm_catalogue = list(zip(storm_catalogue[0], storm_catalogue[1])) # convert to list of tuples

In [17]:
# create a pandas dataframe to store the data
import pandas as pd

df = pd.DataFrame(storm_catalogue, columns=['begin', 'end'])
print('There are {} storms in the catalogue'.format(len(df)))

df.to_csv('../hrrr17x_catalogue_contiguous.csv', index=False)

There are 345 storms in the catalogue
